<a href="https://colab.research.google.com/github/tommymmcguire/Duke-AI-MEng-chatbot/blob/main/notebooks/finetune_mistral_instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup Lit GPT

In [1]:
!pip install 'litgpt[all] @ git+https://github.com/Lightning-AI/litgpt'

  Cloning https://github.com/Lightning-AI/litgpt to /tmp/pip-install-ml907r1x/litgpt_cba2383648054e7d89f1e2272aa0446a
  Running command git clone --filter=blob:none --quiet https://github.com/Lightning-AI/litgpt /tmp/pip-install-ml907r1x/litgpt_cba2383648054e7d89f1e2272aa0446a
  Resolved https://github.com/Lightning-AI/litgpt to commit c81800f455dd997f786cbe2e110eff1f5c0d2d3b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.2/192.2 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.7/90.7 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 107.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 67.1 MB/s eta 0:00:

In [ ]:
!pwd

/content


### Download Model - Mistral7B

In [2]:
!litgpt download --repo_id mistralai/Mistral-7B-v0.1
# !litgpt convert_hf_checkpoint --checkpoint_dir checkpoints/mistralai/Mistral-7B

Setting HF_HUB_ENABLE_HF_TRANSFER=1
config.json: 100% 571/571 [00:00<00:00, 2.80MB/s]
generation_config.json: 100% 116/116 [00:00<00:00, 583kB/s]
pytorch_model-00001-of-00002.bin: 100% 9.94G/9.94G [00:25<00:00, 387MB/s]
pytorch_model-00002-of-00002.bin: 100% 5.06G/5.06G [00:17<00:00, 288MB/s]
pytorch_model.bin.index.json: 100% 23.9k/23.9k [00:00<00:00, 99.3MB/s]
tokenizer.json: 100% 1.80M/1.80M [00:00<00:00, 3.52MB/s]
tokenizer.model: 100% 493k/493k [00:00<00:00, 424MB/s]
tokenizer_config.json: 100% 967/967 [00:00<00:00, 7.28MB/s]
Converting checkpoint files to LitGPT format.
Processing checkpoints/mistralai/Mistral-7B-v0.1/pytorch_model-00001-of-00002.bin
Loading 'model.embed_tokens.weight' into RAM
Loading 'model.layers.0.self_attn.o_proj.weight' into RAM
Loading 'model.layers.0.mlp.gate_proj.weight' into RAM
Loading 'model.layers.0.mlp.up_proj.weight' into RAM
Loading 'model.layers.0.mlp.down_proj.weight' into RAM
Loading 'model.layers.0.input_layernorm.weight' into RAM
Loading 'mod

In [ ]:
!litgpt generate base --precision bf16-true --checkpoint_dir checkpoints/mistralai/Mistral-7B-v0.1 --prompt "Are you good at answering Questions?"

Loading model 'checkpoints/mistralai/Mistral-7B-v0.1/lit_model.pth' with {'name': 'Mistral-7B-v0.1', 'hf_config': {'name': 'Mistral-7B-v0.1', 'org': 'mistralai'}, 'scale_embeddings': False, 'block_size': 4096, 'vocab_size': 32000, 'padding_multiple': 512, 'padded_vocab_size': 32000, 'n_layer': 32, 'n_head': 32, 'head_size': 128, 'n_embd': 4096, 'rotary_percentage': 1.0, 'parallel_residual': False, 'bias': False, 'lm_head_bias': False, 'n_query_groups': 8, 'shared_attention_norm': False, 'norm_class_name': 'RMSNorm', 'norm_eps': 1e-05, 'mlp_class_name': 'LLaMAMLP', 'gelu_approximate': 'none', 'intermediate_size': 14336, 'rope_condense_ratio': 1, 'rope_base': 10000, 'n_expert': 0, 'n_expert_per_token': 0, 'rope_n_elem': 128}
Time to instantiate model: 0.32 seconds.
Time to load the model weights: 12.37 seconds.
Seed set to 1234
Are you good at answering Questions? Have you ever wondered how you would do in a Q&A? In this post, I will give you 10 Q&A Questions for you to practice with and

## Get Data

### Finetune Model

In [ ]:
!litgpt finetune lora \
  --checkpoint_dir "checkpoints/mistralai/Mistral-7B-v0.1/" \
  --precision bf16-true \
  --quantize bnb.fp4 \
  --data AlpacaGPT4\
  --out_dir "out/lora_weights/mistral7B-finetuned/" \
  --train.log_interval 16 \
  --train.save_interval 200 \
  --train.micro_batch_size 2 \
  --train.global_batch_size 32 \
  --train.max_seq_length 512 \
  --train.epochs 3 \
  --train.lr_warmup_steps=100

{'checkpoint_dir': PosixPath('checkpoints/mistralai/Mistral-7B-v0.1'),
 'data': AlpacaGPT4(mask_prompt=False,
                    val_split_fraction=0.03847,
                    prompt_style=<litgpt.prompts.Alpaca object at 0x79a72fa71a80>,
                    ignore_index=-100,
                    seed=42,
                    num_workers=4,
                    download_dir=PosixPath('data/alpacagpt4')),
 'devices': 1,
 'eval': EvalArgs(interval=100, max_new_tokens=100, max_iters=100),
 'logger_name': 'csv',
 'lora_alpha': 16,
 'lora_dropout': 0.05,
 'lora_head': False,
 'lora_key': False,
 'lora_mlp': False,
 'lora_projection': False,
 'lora_query': True,
 'lora_r': 8,
 'lora_value': True,
 'out_dir': PosixPath('out/lora_weights/mistral7B-finetuned-test'),
 'precision': 'bf16-true',
 'quantize': 'bnb.fp4',
 'seed': 1337,
 'train': TrainArgs(save_interval=1,
                    log_interval=16,
                    global_batch_size=32,
                    micro_batch_size=2,
          

In [ ]:
!litgpt generate base \
  --checkpoint_dir "checkpoints/instruct-tuned" \
  --prompt f"{prompt_RAG}" \
  --precision bf16-true

Loading model 'checkpoints/instruct-tuned/lit_model.pth' with {'name': 'Mistral-7B-v0.1', 'hf_config': {'name': 'Mistral-7B-v0.1', 'org': 'mistralai'}, 'scale_embeddings': False, 'block_size': 4096, 'vocab_size': 32000, 'padding_multiple': 512, 'padded_vocab_size': 32000, 'n_layer': 32, 'n_head': 32, 'head_size': 128, 'n_embd': 4096, 'rotary_percentage': 1.0, 'parallel_residual': False, 'bias': False, 'lm_head_bias': False, 'n_query_groups': 8, 'shared_attention_norm': False, 'norm_class_name': 'RMSNorm', 'norm_eps': 1e-05, 'mlp_class_name': 'LLaMAMLP', 'gelu_approximate': 'none', 'intermediate_size': 14336, 'rope_condense_ratio': 1, 'rope_base': 10000, 'n_expert': 0, 'n_expert_per_token': 0, 'rope_n_elem': 128}
Time to instantiate model: 0.15 seconds.
Time to load the model weights: 12.26 seconds.
Seed set to 1234
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
fCHUNK: FLEXIBILITY AND OPTIONS 4+1: BSE+Master's

In [ ]:
prompt_RAG = "CHUNK: FLEXIBILITY AND OPTIONS \
4+1: BSE+Master's Option for Duke Undergraduates \
Duke undergraduate students can complete undergrad and this master's degree in just five (5) years. \
Scholarship opportunity: The AI 4+1 BSE+Master's scholarship covers 20 percent of the costs. Eligibility and other conditions apply. \
MD+MEng in Artificial Intelligence Dual Degree \
Medical students at Duke can complete this degree during their third year. See Duke MEDx website and School of Medicine bulletin for details.\
Scholarship opportunity: The MD+MEng AI scholarship covers 20 percent of the costs. Eligibility and other conditions apply.\
=== \
CURRICULUM SCHEDULES \
The core of the curriculum follows a cohort-based course sequence \
On-Campus Accelerated Option: 12 Months\
Summer:\
Pre-requisite \
AIPI 503: Python & Data Science Math Bootcamp \
Fall: \
AIPI 510: Sourcing Data for Analytics \
AIPI 520: Modeling Process & Algorithms \
AIPI Departmental Elective \
MENG 570: Business Fundamentals for Engineers \
AIPI 501: Industry Seminar Series \
EGR 590: Career Strategy & Design \
====== \
prompt: Is is possible to complete the Meng AI program in just 1 more year after undergrad and what are prerequisites?"

### Merging LoRA Weights

In [3]:
import os
os.mkdir("checkpoints/instruct-tuned")
os.mkdir("checkpoints/instruct-tuned-hf")

In [4]:
!unzip drive/MyDrive/instruct-alpaca.zip -d checkpoints/instruct-tuned

Archive:  drive/MyDrive/instruct-alpaca.zip
  inflating: checkpoints/instruct-tuned/lit_model.pth.lora  
  inflating: checkpoints/instruct-tuned/tokenizer_config.json  
  inflating: checkpoints/instruct-tuned/model_config.yaml  
  inflating: checkpoints/instruct-tuned/tokenizer.json  
  inflating: checkpoints/instruct-tuned/generation_config.json  
  inflating: checkpoints/instruct-tuned/hyperparameters.yaml  
 extracting: checkpoints/instruct-tuned/prompt_style.yaml  
  inflating: checkpoints/instruct-tuned/tokenizer.model  
   creating: checkpoints/instruct-tuned/.ipynb_checkpoints/


In [5]:
!litgpt merge_lora \
 --checkpoint_dir "checkpoints/instruct-tuned" \
 --pretrained_checkpoint_dir "checkpoints/mistralai/Mistral-7B-v0.1" \
 --precision bf16-true

Saved merged weights to 'checkpoints/instruct-tuned/lit_model.pth'


### Zip and Save

In [29]:
import os
os.chdir("/content/checkpoints/")
!zip -r custom-instruct-mistral7B-hf.zip custom-instruct-mistral7B
os.chdir("/content/")
!pwd

  adding: custom-instruct-mistral7B/ (stored 0%)
  adding: custom-instruct-mistral7B/tokenizer.model (deflated 55%)
  adding: custom-instruct-mistral7B/model-00003-of-00006.safetensors


zip error: Interrupted (aborting)
/content


In [ ]:
!zip -r mistral7B-finetuned-alpaca-final.zip out/lora_weights/mistral7B-finetuned-

  adding: out/lora_weights/mistral7B-finetuned-test/step-000001/ (stored 0%)
  adding: out/lora_weights/mistral7B-finetuned-test/step-000001/lit_model.pth.lora (deflated 52%)
  adding: out/lora_weights/mistral7B-finetuned-test/step-000001/tokenizer_config.json (deflated 68%)
  adding: out/lora_weights/mistral7B-finetuned-test/step-000001/model_config.yaml (deflated 42%)
  adding: out/lora_weights/mistral7B-finetuned-test/step-000001/tokenizer.json (deflated 74%)
  adding: out/lora_weights/mistral7B-finetuned-test/step-000001/generation_config.json (deflated 22%)
  adding: out/lora_weights/mistral7B-finetuned-test/step-000001/hyperparameters.yaml (deflated 46%)
  adding: out/lora_weights/mistral7B-finetuned-test/step-000001/prompt_style.yaml (stored 0%)
  adding: out/lora_weights/mistral7B-finetuned-test/step-000001/tokenizer.model (deflated 55%)


In [ ]:
!cp mistral7B-finetuned-alpaca-final.zip /content/drive/MyDrive/

cp: cannot stat 'mistral7B-finetuned-logic-final.zip': No such file or directory


In [ ]:
!unzip instruct-alpaca.zip -d checkpoints/Mistral-instruct-alpaca

Archive:  instruct-alpaca.zip
  inflating: checkpoints/Mistral-instruct-alpaca/lit_model.pth.lora  
  inflating: checkpoints/Mistral-instruct-alpaca/tokenizer_config.json  
  inflating: checkpoints/Mistral-instruct-alpaca/model_config.yaml  
  inflating: checkpoints/Mistral-instruct-alpaca/tokenizer.json  
  inflating: checkpoints/Mistral-instruct-alpaca/generation_config.json  
  inflating: checkpoints/Mistral-instruct-alpaca/hyperparameters.yaml  
 extracting: checkpoints/Mistral-instruct-alpaca/prompt_style.yaml  
  inflating: checkpoints/Mistral-instruct-alpaca/tokenizer.model  
   creating: checkpoints/Mistral-instruct-alpaca/.ipynb_checkpoints/


### Convert to Huggingface

In [6]:
!litgpt convert from_litgpt \
    --checkpoint_dir checkpoints/instruct-tuned \
    --output_dir checkpoints/instruct-tuned-hf

Loading 'lm_head.weight' into RAM
Loading 'transformer.wte.weight' into RAM
Loading 'transformer.h.0.norm_1.weight' into RAM
Loading 'transformer.h.0.attn.attn.weight' into RAM
Loading 'transformer.h.0.attn.proj.weight' into RAM
Loading 'transformer.h.0.norm_2.weight' into RAM
Loading 'transformer.h.0.mlp.fc_1.weight' into RAM
Loading 'transformer.h.0.mlp.fc_2.weight' into RAM
Loading 'transformer.h.0.mlp.proj.weight' into RAM
Loading 'transformer.h.1.norm_1.weight' into RAM
Loading 'transformer.h.1.attn.attn.weight' into RAM
Loading 'transformer.h.1.attn.proj.weight' into RAM
Loading 'transformer.h.1.norm_2.weight' into RAM
Loading 'transformer.h.1.mlp.fc_1.weight' into RAM
Loading 'transformer.h.1.mlp.fc_2.weight' into RAM
Loading 'transformer.h.1.mlp.proj.weight' into RAM
Loading 'transformer.h.2.norm_1.weight' into RAM
Loading 'transformer.h.2.attn.attn.weight' into RAM
Loading 'transformer.h.2.attn.proj.weight' into RAM
Loading 'transformer.h.2.norm_2.weight' into RAM
Loading 'tra

In [7]:
from huggingface_hub import notebook_login

notebook_login()

In [8]:
!cp checkpoints/mistralai/Mistral-7B-v0.1/config.json checkpoints/instruct-tuned-hf/config.json

In [20]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_8bit=True)
state_dict = torch.load('checkpoints/instruct-tuned-hf/model.pth')
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1",
                                  state_dict=state_dict,
                                  device_map="auto",
                                  quantization_config=quantization_config
                                  )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [22]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

In [25]:
from typing import Any, Dict
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

dtype = torch.bfloat16 if torch.cuda.get_device_capability()[0] == 8 else torch.float16

class EndpointHandler:
    def __init__(self, path="mkeohane01/mistral-instruct-590"):
        tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code = True)
        model = AutoModelForCausalLM.from_pretrained(
            path,
            return_dict = True,
            device_map = "auto",
            load_in_8bit = True,
            torch_dtype = dtype,
            trust_remote_code = True,
        )

        gen_config = model.generation_config
        gen_config.max_new_tokens = 256
        gen_config.temperature = 0.7
        gen_config.num_return_sequences = 1
        gen_config.pad_token_id = tokenizer.eos_token_id
        gen_config.eos_token_id = tokenizer.eos_token_id

        self.generation_config = gen_config

        self.pipeline = pipeline(
            'text-generation', model=model, tokenizer=tokenizer
        )



    def __call__(self, data: Dict[dict, Any]) -> Dict[str, Any]:
        prompt = data.pop("inputs", data)

        system_message = "Use the provided context to answer question or follow the instruction."

        full_prompt = f"""<s>### Instruction:
        {system_message}
        ### Input:
        {prompt}
        ### Response:
        """

        result = self.pipeline(full_prompt, generation_config = self.generation_config)[0]['generated_text']

        # if re.search(r'### Response:(.*?)###', result, re.DOTALL):
        #     result =  re.search(r'### Response:(.*?)###', result, re.DOTALL).group(1).strip()

        # elif re.search(r"### Response:(.*)", result, re.DOTALL):
        #     result = re.search(r"### Response:(.*)", result, re.DOTALL).group(1).strip()

        return result

In [26]:
handler = EndpointHandler()
result = handler({"inputs": "Can I complete the Duke University AI Masters of Engineering program in 1 year?"})
print(result)

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:155: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model-00002-of-00002.safetensors:   0%|          | 0.00/2.56G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


<s>### Instruction:
        Use the provided context followed by a question to answer it.
        ### Input:
        Can I complete the Duke University AI Masters of Engineering program in 1 year?
        ### Response: 
        1. Yes, you can complete the Duke University AI Masters of Engineering program in 1 year.
        2. No, you cannot complete the Duke University AI Masters of Engineering program in 1 year.
        3. It depends on your schedule and the availability of the program.
        4. You can complete the Duke University AI Masters of Engineering program in 1 year if you are a full-time student.
        5. You can complete the Duke University AI Masters of Engineering program in 1 year if you are a part-time student.
        6. You can complete the Duke University AI Masters of Engineering program in 1 year if you are a distance learning student.
        7. You can complete the Duke University AI Masters of Engineering program in 1 year if you are a full-time student and

In [16]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load model and tokenizer
# tokenizer = AutoTokenizer.from_pretrained("gpt2")
# model = AutoModelForCausalLM.from_pretrained("gpt2")

# Input text
input_text = "If I give you a chunk with information, can you give me info on Duke AI Masters program?"
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Generate response
output_ids = model.generate(input_ids, max_length=50)

# Decode and print output
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(output_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


If I give you a chunk with information, can you give me info on Duke AI Masters program?

I am a 2018 graduate of the University of Michigan with a B.S. in Computer Science. I have been


In [27]:
tokenizer.save_pretrained("checkpoints/custom-instruct-mistral7B")

('checkpoints/custom-instruct-mistral7B/tokenizer_config.json',
 'checkpoints/custom-instruct-mistral7B/special_tokens_map.json',
 'checkpoints/custom-instruct-mistral7B/tokenizer.model',
 'checkpoints/custom-instruct-mistral7B/added_tokens.json',
 'checkpoints/custom-instruct-mistral7B/tokenizer.json')

In [25]:
model.save_pretrained("checkpoints/custom-instruct-mistral7B")

In [9]:
model.push_to_hub("mkeohane01/mistral-instruct-590")
tokenizer.push_to_hub("mkeohane01/mistral-instruct-590")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Removed shared tensor {'model.layers.8.self_attn.k_proj.weight_format', 'model.layers.15.mlp.down_proj.weight_format', 'model.layers.16.mlp.up_proj.weight_format', 'model.layers.17.mlp.down_proj.weight_format', 'model.layers.5.mlp.down_proj.weight_format', 'model.layers.7.mlp.down_proj.weight_format', 'model.layers.2.self_attn.v_proj.weight_format', 'model.layers.28.self_attn.o_proj.weight_format', 'model.layers.30.self_attn.o_proj.weight_format', 'model.layers.1.mlp.up_proj.weight_format', 'model.layers.24.mlp.gate_proj.weight_format', 'model.layers.9.mlp.down_proj.weight_format', 'model.layers.13.mlp.down_proj.weight_format', 'model.layers.25.self_attn.k_proj.weight_format', 'model.layers.31.self_attn.o_proj.weight_format', 'model.layers.16.self_attn.o_proj.weight_format', 'model.layers.0.mlp.gate_proj.weight_format', 'model.layers.30.mlp.gate_proj.weight_format', 'model.layers.16.mlp.down_proj.weight_format', 'model.layers.11.self_attn.k_proj.weight_format', 'model.layers.17.self_at

model-00002-of-00002.safetensors:   0%|          | 0.00/2.56G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mkeohane01/mistral-instruct-590/commit/f29946dfe99579e917b85239839f2c413ed2e00c', commit_message='Upload tokenizer', commit_description='', oid='f29946dfe99579e917b85239839f2c413ed2e00c', pr_url=None, pr_revision=None, pr_num=None)

In [19]:
import gc
gc.collect()
torch.cuda.empty_cache()